In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import os
from io import StringIO

cwd = os.getcwd() # Get direction

In [2]:
import json
import requests
from urllib.request import urlopen
import plotly.express as px

### Function to generate dataframe from USDA API

In [3]:
# http direction 
nass_base = 'http://quickstats.nass.usda.gov/api/api_GET?'
# My personal key
key= '1897574B-86B2-3182-B9C3-3BFF302E3042'

def api_nass(api_params, key):
    apidata = requests.get(nass_base, params=api_params)
    apidata = apidata.json()
    df = pd.DataFrame.from_dict(apidata['data'])
    return df

### Function to append especific variables for several years by "area operated" from Ag Census. 
#### Returns dataframe with time series format

In [4]:
def dataframe_usda(varlist,key, years, sector ,commodity,domain,geolevel, separ, label_index):
    dfA = pd.DataFrame(columns=['year','state_fips_code','state_name','domaincat_desc'])

    for z in varlist:
        label= z.split(separ)[label_index]
        dfB = pd.DataFrame()
        for y in years:
            # Parameters for land tenure data
            area_params = {'key': key,
                       'sector_desc': sector,
                       'commodity_desc': commodity,
                       'domain_desc': domain,
                       'short_desc': z,
                       'agg_level_desc': geolevel.upper(),
                       'year': y}
            df= api_nass(area_params, key)
            df[label]= df['Value'].str.replace(',', '')
            df[label]= pd.to_numeric(df[label], errors='coerce')
            df[label].replace(np.nan, 'D', inplace=True) #These are rows without enough obs 'D'

            # merge number of operations (farms) with area

            filt=['year','state_name','state_fips_code','domaincat_desc',label]

            dfB=dfB.append(df[filt], 'sort=False') #append years

        dfA=dfA.merge(dfB, how='outer', on=['year','state_fips_code','state_name','domaincat_desc']) #merge vars          

    return dfA

### Data on precipitation-adaptation practices

In [6]:
pract_acre=['PRACTICES, LAND USE, CROPLAND, COVER CROP PLANTED, (EXCL CRP) - ACRES',
           'PRACTICES, LAND USE, DRAINED BY TILE - ACRES']

pract_ope=['PRACTICES, LAND USE, CROPLAND, COVER CROP PLANTED, (EXCL CRP) - NUMBER OF OPERATIONS',
          'PRACTICES, LAND USE, DRAINED BY TILE - NUMBER OF OPERATIONS']
dfprac= dataframe_usda(pract_acre+pract_ope,key,[2012,2017,2022],'ECONOMICS','PRACTICES','AREA OPERATED','state',separ=', ',label_index=-1) #dataframe of practices in agriculture

In [7]:
dfprac= dfprac[dfprac['state_name']!= 'ALASKA']
dfprac= dfprac[dfprac['state_name']!= 'HAWAII']
dfprac.replace(np.nan, 0, inplace=True)

In [8]:
dfprac

,year,state_name,state_fips_code,domaincat_desc,(EXCL CRP) - ACRES,DRAINED BY TILE - ACRES,(EXCL CRP) - NUMBER OF OPERATIONS,DRAINED BY TILE - NUMBER OF OPERATIONS
0,2012,ALABAMA,01,"AREA OPERATED: (1,000 TO 1,999 ACRES)",51652,8750,148.0,69.0
2,2012,ARIZONA,04,"AREA OPERATED: (1,000 TO 1,999 ACRES)",3908,D,13.0,6.0
3,2012,ARKANSAS,05,"AREA OPERATED: (1,000 TO 1,999 ACRES)",34967,121994,85.0,123.0
4,2012,CALIFORNIA,06,"AREA OPERATED: (1,000 TO 1,999 ACRES)",56062,77528,140.0,135.0
5,2012,COLORADO,08,"AREA OPERATED: (1,000 TO 1,999 ACRES)",27217,15089,144.0,54.0
...,...,...,...,...,...,...,...,...
1784,2022,WISCONSIN,55,AREA OPERATED: (70.0 TO 99.9 ACRES),10744,15895,594.0,505.0
1785,2022,WYOMING,56,AREA OPERATED: (70.0 TO 99.9 ACRES),638,817,20.0,19.0
1786,2012,MASSACHUSETTS,25,"AREA OPERATED: (2,000 OR MORE ACRES)",0,D,0.0,1.0
1788,2017,MASSACHUSETTS,25,"AREA OPERATED: (2,000 OR MORE ACRES)",0,D,0.0,1.0


In [9]:
A= ['(EXCL CRP) - ACRES',
     'DRAINED BY TILE - ACRES',
    '(EXCL CRP) - NUMBER OF OPERATIONS',
       'DRAINED BY TILE - NUMBER OF OPERATIONS']
df=dfprac[A]
df.dropna(how='all')

,(EXCL CRP) - ACRES,DRAINED BY TILE - ACRES,(EXCL CRP) - NUMBER OF OPERATIONS,DRAINED BY TILE - NUMBER OF OPERATIONS
0,51652,8750,148.0,69.0
2,3908,D,13.0,6.0
3,34967,121994,85.0,123.0
4,56062,77528,140.0,135.0
5,27217,15089,144.0,54.0
...,...,...,...,...
1784,10744,15895,594.0,505.0
1785,638,817,20.0,19.0
1786,0,D,0.0,1.0
1788,0,D,0.0,1.0


### Data of land size categories

In [70]:
#cat=['AG LAND, CROPLAND, HARVESTED - ACRES','AG LAND, CROPLAND, HARVESTED - NUMBER OF OPERATIONS']
#dfland= dataframe_usda(cat,key, [2012,2017,2022], 'ECONOMICS','AG LAND','AREA OPERATED','state',separ='- ',label_index=-1)
cat= ['FARM OPERATIONS - ACRES OPERATED', 'FARM OPERATIONS - NUMBER OF OPERATIONS']
dfland= dataframe_usda(cat,key, [2012,2017,2022], 'ECONOMICS','FARM OPERATIONS','AREA OPERATED','state',separ='- ',label_index=-1)


In [71]:
## Clean data
#Select categories
selection=['AREA OPERATED: (1,000 TO 1,999 ACRES)',
       'AREA OPERATED: (1.0 TO 9.9 ACRES)',
       'AREA OPERATED: (10.0 TO 49.9 ACRES)',
       'AREA OPERATED: (100 TO 139 ACRES)',
       'AREA OPERATED: (140 TO 179 ACRES)',
       'AREA OPERATED: (180 TO 219 ACRES)',
       'AREA OPERATED: (2,000 OR MORE ACRES)',
       'AREA OPERATED: (220 TO 259 ACRES)',
       'AREA OPERATED: (260 TO 499 ACRES)',
       'AREA OPERATED: (50.0 TO 69.9 ACRES)',
       'AREA OPERATED: (500 TO 999 ACRES)',
       'AREA OPERATED: (70.0 TO 99.9 ACRES)']
# New dataset with selected categories of land size    
dfland1= dfland.loc[dfland['domaincat_desc'].isin(selection)]
#### Define upper and lower bounds of land size tabulated data
# Extract numbers (acres) from column: categories description
dfland1['farmcat']=dfland1['domaincat_desc']
dfland1['farmcat']= dfland1['farmcat'].str.replace(',', '', regex=True)
dfland1['farmcat']= dfland1['farmcat'].str.replace('.0', '', regex=False)
dfland1['farmcat']= dfland1['farmcat'].str.replace('.9', '', regex=False)

dfs = dfland1['farmcat'].str.findall('\d+').to_frame()
dfland1['lower_bound']= dfs.apply(lambda x: x.str[0])
dfland1['upper_bound']= dfs.apply(lambda x: x.str[1])

dfland1['lower_bound']=dfland1['lower_bound'].astype(int)
dfland1['upper_bound']=pd.to_numeric(dfland1['upper_bound'], errors='coerce')
#dfland1['upper_bound'].astype(int)


/Users/elizabethespinosa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/elizabethespinosa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/elizabethespinosa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [72]:
dfland1= dfland1[dfland1['state_name']!= 'ALASKA']
dfland1= dfland1[dfland1['state_name']!= 'HAWAII']

### Data on policies in terms of land area

In [12]:
varlist= ['AG LAND, CROP INSURANCE - ACRES','AG LAND, CROP INSURANCE - NUMBER OF OPERATIONS','AG LAND, IRRIGATED - NUMBER OF OPERATIONS',
         'AG LAND, RENTED FROM OTHERS, IN FARMS - ACRES','AG LAND, RENTED FROM OTHERS, IN FARMS - NUMBER OF OPERATIONS',
         'AG LAND, OWNED, IN FARMS - ACRES','AG LAND, OWNED, IN FARMS - NUMBER OF OPERATIONS']
dfcrop= dataframe_usda(varlist,key, [2012,2017,2022], 'ECONOMICS','AG LAND','AREA OPERATED','state',separ=', ',label_index=1)


In [13]:
## irrigated land in acres (only cropland)
varlist= ['AG LAND, CROPLAND, HARVESTED, IRRIGATED - ACRES','AG LAND, CROPLAND, HARVESTED, IRRIGATED - NUMBER OF OPERATIONS']
dfirr= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','AG LAND','AREA OPERATED','state',separ=', ',label_index=3)


###  Data on business operations: income and expenses

In [4]:
## Variable list of farm operations grouped by 'commodity(labor, feed, etc)' description
varlist = [['INCOME, NET CASH FARM, OF OPERATIONS - NET INCOME, MEASURED IN $ / OPERATION'], #net income
       ['GOVT PROGRAMS, FEDERAL - RECEIPTS, MEASURED IN $','GOVT PROGRAMS, FEDERAL - OPERATIONS WITH RECEIPTS'], #federal programs
       ['LABOR, HIRED - EXPENSE, MEASURED IN $'],#labor
          ['INCOME, FARM-RELATED - RECEIPTS, MEASURED IN $','INCOME, FARM-RELATED - OPERATIONS WITH RECEIPTS', # farm-related income
          'INCOME, FARM-RELATED, GOVT PROGRAMS, STATE & LOCAL - RECEIPTS, MEASURED IN $', # local and state government transfers
           'INCOME, FARM-RELATED, GOVT PROGRAMS, STATE & LOCAL - OPERATIONS WITH RECEIPTS']
          #['FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN $', #fertilizer
           #'FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE'],
        #"LABOR, CONTRACT - EXPENSE, MEASURED IN $",'LABOR, CONTRACT - OPERATIONS WITH EXPENSE',
           #['SEEDS & PLANTS TOTALS - EXPENSE, MEASURED IN $','SEEDS & PLANTS TOTALS - OPERATIONS WITH EXPENSE'],
       #['RENT, CASH, LAND & BUILDINGS - EXPENSE, MEASURED IN $',"RENT, CASH, LAND & BUILDINGS - OPERATIONS WITH EXPENSE"]
      #['FUELS, INCL LUBRICANTS - EXPENSE, MEASURED IN $','FUELS, INCL LUBRICANTS - OPERATIONS WITH EXPENSE'],
      #['EXPENSE TOTALS, OPERATING - EXPENSE, MEASURED IN $','EXPENSE TOTALS, OPERATING - OPERATIONS WITH EXPENSE'],
      #['CHEMICAL TOTALS - EXPENSE, MEASURED IN $','CHEMICAL TOTALS - OPERATIONS WITH EXPENSE'],
        #['FEED - OPERATIONS WITH EXPENSE',],
      #['EXPENSE TOTALS, OPERATING - EXPENSE, MEASURED IN $ / OPERATION'],
      #['CCC LOANS - RECEIPTS, MEASURED IN $','CCC LOANS - OPERATIONS WITH RECEIPTS']
          ]

commodity = ['INCOME, NET CASH FARM','GOVT PROGRAMS','LABOR','INCOME, FARM-RELATED']
             #,'CCC LOANS','LABOR','SEEDS & PLANTS TOTALS','RENT','FUELS','EXPENSE TOTALS','CHEMICAL TOTALS','FEED','FERTILIZER TOTALS','EXPENSE TOTALS',]


In [15]:
varlist = ['INCOME, NET CASH FARM, OF OPERATIONS - NET INCOME, MEASURED IN $ / OPERATION']
dfy= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','INCOME, NET CASH FARM','AREA OPERATED','state',separ='- ',label_index=1)


In [16]:
varlist = ['GOVT PROGRAMS, FEDERAL - RECEIPTS, MEASURED IN $','GOVT PROGRAMS, FEDERAL - OPERATIONS WITH RECEIPTS']
dfg= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','GOVT PROGRAMS','AREA OPERATED','state',separ=', ',label_index=1)


In [17]:
varlist = ['INCOME, FARM-RELATED - RECEIPTS, MEASURED IN $','INCOME, FARM-RELATED - OPERATIONS WITH RECEIPTS', # farm-related income
          ]
dfyf= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','INCOME, FARM-RELATED','AREA OPERATED','state',separ=', ',label_index=1)


In [18]:
varlist= ['INCOME, FARM-RELATED, GOVT PROGRAMS, STATE & LOCAL - RECEIPTS, MEASURED IN $', # local and state government transfers
           'INCOME, FARM-RELATED, GOVT PROGRAMS, STATE & LOCAL - OPERATIONS WITH RECEIPTS']
dfyf1= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','INCOME, FARM-RELATED','AREA OPERATED','state',separ=', ',label_index=3)


In [19]:
varlist=['LABOR, HIRED - EXPENSE, MEASURED IN $', 'LABOR, HIRED - OPERATIONS WITH EXPENSE',
       "LABOR, CONTRACT - EXPENSE, MEASURED IN $",'LABOR, CONTRACT - OPERATIONS WITH EXPENSE']
dfl= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','LABOR','AREA OPERATED','state',separ=', ',label_index=1)


In [20]:
varlist= ['RENT, CASH, LAND & BUILDINGS - EXPENSE, MEASURED IN $',"RENT, CASH, LAND & BUILDINGS - OPERATIONS WITH EXPENSE"]
dfre= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','RENT','AREA OPERATED','state',separ=', ',label_index=2)


In [21]:
varlist= ['EXPENSE TOTALS, OPERATING - EXPENSE, MEASURED IN $','EXPENSE TOTALS, OPERATING - OPERATIONS WITH EXPENSE']
dfex= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','EXPENSE TOTALS','AREA OPERATED','state',separ='- ',label_index=1)


In [22]:
dfre

,year,state_name,state_fips_code,domaincat_desc,LAND & BUILDINGS - EXPENSE,LAND & BUILDINGS - OPERATIONS WITH EXPENSE
0,2012,ALABAMA,01,"AREA OPERATED: (1,000 TO 1,999 ACRES)",2.3078e+07,507
1,2012,ALASKA,02,"AREA OPERATED: (1,000 TO 1,999 ACRES)",61000,7
2,2012,ARIZONA,04,"AREA OPERATED: (1,000 TO 1,999 ACRES)",2.5654e+07,250
3,2012,ARKANSAS,05,"AREA OPERATED: (1,000 TO 1,999 ACRES)",5.4095e+07,993
4,2012,CALIFORNIA,06,"AREA OPERATED: (1,000 TO 1,999 ACRES)",2.43963e+08,1294
...,...,...,...,...,...,...
1789,2022,VIRGINIA,51,AREA OPERATED: (70.0 TO 99.9 ACRES),3.478e+06,524
1790,2022,WASHINGTON,53,AREA OPERATED: (70.0 TO 99.9 ACRES),5.479e+06,263
1791,2022,WEST VIRGINIA,54,AREA OPERATED: (70.0 TO 99.9 ACRES),351000,128
1792,2022,WISCONSIN,55,AREA OPERATED: (70.0 TO 99.9 ACRES),9.191e+06,1019


### Data on access to machinery

In [23]:
varlist= ['MACHINERY TOTALS - ASSET VALUE, MEASURED IN $', 'MACHINERY TOTALS - OPERATIONS WITH ASSET VALUE']
dfmach= dataframe_usda(varlist,key, [2012,2017, 2022], 'ECONOMICS','MACHINERY TOTALS','AREA OPERATED','state',separ=' - ',label_index=0)


### Data on operations by NAICS and farm size

In [24]:
varlist= ['FARM OPERATIONS - NUMBER OF OPERATIONS']
dfnaics= dataframe_usda(varlist,key,[2012,2017, 2022], 'ECONOMICS','FARM OPERATIONS','AREA OPERATED AND NAICS CLASSIFICATION','state',separ=' - ',label_index=0)

In [25]:
dfnaics['domain']= dfnaics['domaincat_desc']
dfnaics['naics']= dfnaics['domain'].str.split(' AND ').str[1]
dfnaics['domaincat_desc']= dfnaics['domain'].str.split(' AND ').str[0]


In [28]:

## Quantify operations practicing main agricultural activities
dfnaics['grains']= np.where(dfnaics['naics']=='NAICS CLASSIFICATION: (1111)', 1, 0)
dfnaics['veggies']= np.where(dfnaics['naics']=='NAICS CLASSIFICATION: (1112)', 1, 0)
dfnaics['fruit']= np.where(dfnaics['naics']=='NAICS CLASSIFICATION: (1113)', 1, 0)
dfnaics['cattle']= np.where(dfnaics['naics']=='NAICS CLASSIFICATION: (112111)', 1, 0)

for x in ['grains','veggies','fruit','cattle']:
    dfnaics[x] = dfnaics[x]* dfnaics['FARM OPERATIONS'] #get number of farms in each agric code

dfic= dfnaics.groupby(['year','state_fips_code','domaincat_desc'])['grains','veggies','fruit','cattle'].max().reset_index()


In [29]:
dfic

,year,state_fips_code,domaincat_desc,grains,veggies,fruit,cattle
0,2012,01,"AREA OPERATED: (1,000 TO 1,999 ACRES)",127,4,13,291
1,2012,01,AREA OPERATED: (1.0 TO 9.9 ACRES),40,233,216,643
2,2012,01,AREA OPERATED: (10.0 TO 49.9 ACRES),327,475,489,4959
3,2012,01,AREA OPERATED: (100 TO 139 ACRES),148,83,106,2013
4,2012,01,AREA OPERATED: (140 TO 179 ACRES),136,45,78,1340
...,...,...,...,...,...,...,...
1795,2022,56,AREA OPERATED: (220 TO 259 ACRES),9,0,0,85
1796,2022,56,AREA OPERATED: (260 TO 499 ACRES),75,1,0,350
1797,2022,56,AREA OPERATED: (50.0 TO 69.9 ACRES),13,2,0,128
1798,2022,56,AREA OPERATED: (500 TO 999 ACRES),63,0,1,406


### Weather and Climate Data NOAA

In [3]:
def annual_climate(var, year1, year2, month1, month2):
    dfy = pd.DataFrame(columns=['county_fips','year']) #year dataframe
    for ye in np.arange(year1, year2+1,1): #Year range
        dfm= pd.DataFrame() # montly dataframe
        for mo in np.arange(month1, month2+1,1): #Month range
            df= pd.read_csv('https://www.ncei.noaa.gov/data/nclimgrid-daily/access/averages/'+str(ye)+'/'+var+'-'+str(ye)+str(mo).zfill(2)+'-cty-scaled.csv',sep=',', header=None)
            df.rename(columns={1: 'county_fips', 4: 'month'}, inplace=True)
            df.drop(columns=[0,2,3,5], inplace=True)
            dfv = df.melt(id_vars=['county_fips','month'], var_name='days', value_name= var)
            dfm=pd.concat([dfm, dfv], ignore_index=True) #append monthly data

        dfm.replace(-999.99, np.nan, inplace=True) #days of the month < 31 are filled with -999.99
        dfm.dropna(axis=0, inplace=True) #remove day 31 of the month without values
        if var== 'prcp': 
            # Accumulated precipitation
            dfm['pcp_mm']= dfm.groupby('county_fips')['prcp'].transform('sum')
            dfm['pcp_mean']= dfm.groupby('county_fips')['prcp'].transform('mean')
            dfm['pcp_std']= dfm.groupby('county_fips')['prcp'].transform('std')
            # 1-inch pcp days:
            dfm['days'] = np.where(dfm['prcp'] >25.4, 1, 0) #select days with more than 1 inch or 25.4 mm pcp
            dfm['pcp_days'] = dfm.groupby('county_fips')['days'].transform('sum') # accumulated sum 1-inch pcp days in given month
            dfm['pcp_days_prob'] = dfm.groupby('county_fips')['days'].transform('mean')
            
            dfm.drop_duplicates(subset=['county_fips'], keep='first', inplace=True)
            dfm= dfm[['county_fips','pcp_mm','pcp_mean','pcp_std','pcp_days', 'pcp_days_prob']]
            
        elif var== 'tavg': # Average temperature
            dfm['temp']= dfm.groupby('county_fips')['tavg'].transform('mean')
            dfm['temp_std']= dfm.groupby('county_fips')['tavg'].transform('std')
            # tmax > 29C
            dfm['days'] = np.where(dfm['tavg'] >29, 1, 0) #select days with more than 29C
            dfm['edd'] = dfm.groupby('county_fips')['days'].transform('sum') # accumulated sum degree days in given month
            dfm['edd_prob'] = dfm.groupby('county_fips')['days'].transform('mean')
            dfm.drop_duplicates(subset=['county_fips'], keep='first', inplace=True)
            dfm= dfm[['county_fips','temp','temp_std','edd','edd_prob']]
            
        dfm['year']= ye
        dfy= pd.concat([dfy, dfm], ignore_index=True)
    
    return dfy

In [4]:
year1= 1977
year2= 2022
month1= 4
month2= 9
#### Average temperature, max temperature, prcp
df_tavg= annual_climate('tavg',year1, year2, month1, month2)
df_pcp= annual_climate('prcp',year1, year2, month1, month2)

/Users/elizabethespinosa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [30]:
dfh= pd.merge(df_tavg, df_pcp)

In [31]:
# Define time windows 
# Baseline: 1987-2007
# 3 periods: 2008-2012; 2013-2017; 2018-2022
condlist= [(dfh['year']<=2007), 
           ((dfh['year']>2007) & (dfh['year']<2013)), 
          ((dfh['year']>2012) & (dfh['year']<2018)), 
           (dfh['year']>2017)]
choicelist=[0, 1, 2, 3]
dfh['base'] = np.select(condlist, choicelist, np.nan)


In [32]:
# conty codes
dfh['county_fips']= dfh['county_fips'].astype(str)
dfh['county_fips']= dfh['county_fips'].str.zfill(5)
dfh['state_fips']= dfh['county_fips'].str[:-3]
dfh['county_ansi']= dfh['county_fips'].str[-3:]

In [33]:
# Recode states and counties
condlist= [(dfh['state_fips']== '01'), (dfh['state_fips']== '02'), (dfh['state_fips']== '03'), (dfh['state_fips']== '04'),
          (dfh['state_fips']== '05'), (dfh['state_fips']== '06'), (dfh['state_fips']== '07'), (dfh['state_fips']== '08'),
           (dfh['state_fips']== '09'), (dfh['state_fips']== '10'), (dfh['state_fips']== '11'), (dfh['state_fips']== '12'),
           (dfh['state_fips']== '13'), (dfh['state_fips']== '14'), (dfh['state_fips']== '15'), (dfh['state_fips']== '16'),
           (dfh['state_fips']== '17'), (dfh['state_fips']== '18'), (dfh['state_fips']== '19'), (dfh['state_fips']== '20'),
           (dfh['state_fips']== '21'), (dfh['state_fips']== '22'), (dfh['state_fips']== '23'), (dfh['state_fips']== '24'),
           (dfh['state_fips']== '25'), (dfh['state_fips']== '26'), (dfh['state_fips']== '27'), (dfh['state_fips']== '28'),
           (dfh['state_fips']== '29'), (dfh['state_fips']== '30'), (dfh['state_fips']== '31'), (dfh['state_fips']== '32'),
           (dfh['state_fips']== '33'), (dfh['state_fips']== '34'), (dfh['state_fips']== '35'), (dfh['state_fips']== '36'),
           (dfh['state_fips']== '37'), (dfh['state_fips']== '38'), (dfh['state_fips']== '39'), (dfh['state_fips']== '40'),
           (dfh['state_fips']== '41'), (dfh['state_fips']== '42'), (dfh['state_fips']== '43'), (dfh['state_fips']== '44'),           
           (dfh['state_fips']== '45'), (dfh['state_fips']== '46'), (dfh['state_fips']== '47'), (dfh['state_fips']== '48')          
          ]
choicelist=['01','04','05','06','08','09','10','12','13','16','17','18','19','20','21','22','23','24','25','26',
           '27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47',
           '48','49','50','51','53','54','55','56']
dfh['state_ansi'] = np.select(condlist, choicelist, np.nan)
dfh['county_fips']= dfh['state_ansi']+dfh['county_ansi']


In [36]:
dfh.rename(columns={'temp_std': 'temp_var', 'pcp_std': 'pcp_var'}, inplace=True)

In [37]:
dfh.iloc[:,0:]

,county_fips,edd,edd_prob,temp,temp_var,year,pcp_days,pcp_days_prob,pcp_mean,pcp_mm,pcp_var,base,state_fips,county_ansi,state_ansi
0,01001,8.0,0.043716,24.741311,3.651401,1977,4.0,0.021858,2.743716,502.10,6.058049,0.0,01,001,01
1,01003,7.0,0.038251,25.182951,3.326679,1977,7.0,0.038251,3.930219,719.23,7.427518,0.0,01,003,01
2,01005,6.0,0.032787,24.880437,3.451581,1977,3.0,0.016393,3.197322,585.11,7.350439,0.0,01,005,01
3,01007,5.0,0.027322,23.953497,3.805623,1977,5.0,0.027322,3.138306,574.31,7.966790,0.0,01,007,01
4,01009,5.0,0.027322,23.545792,4.262264,1977,6.0,0.032787,3.973770,727.20,11.209357,0.0,01,009,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142917,56037,0.0,0.000000,13.855574,7.119751,2022,0.0,0.000000,0.863934,158.10,2.014583,3.0,48,037,56
142918,56039,0.0,0.000000,9.047049,7.012791,2022,0.0,0.000000,2.202787,403.11,4.104131,3.0,48,039,56
142919,56041,0.0,0.000000,12.213770,6.726333,2022,0.0,0.000000,0.903115,165.27,1.862651,3.0,48,041,56
142920,56043,0.0,0.000000,15.306995,7.727986,2022,1.0,0.005464,1.137705,208.20,2.927870,3.0,48,043,56


In [88]:
## Compute averages and standard deviations
dfm= dfh.groupby(['state_ansi','base']).agg({'temp':{'tavg_mean': 'mean', 'tavg_std': 'std'},
       'temp_var':{'temp_var_mean': 'mean', 'temp_var_std': 'std'},                                        
       'edd':{'edd_mean': 'mean', 'edd_std': 'std'}, 
       'pcp_var':{'pcp_var_mean': 'mean', 'pcp_var_std': 'std'},
       'pcp_mm':{'pcp_mm_mean': 'mean', 'pcp_mm_std': 'std'},
       'pcp_days':{'pcpdays_mean': 'mean', 'pcpdays_std': 'std'}}).reset_index()
## Clean column names
dfm.columns.set_levels(['state_ansi','base','tavg_mean','tavg_std','temp_var_mean', 'temp_var_std',
                        'edd_mean','edd_std', 'pcp_var_mean', 'pcp_var_std',
                        'pcp_mm_mean', 'pcp_mm_std','pcpdays_mean','pcpdays_std'],level=1,inplace=True)
dfm.columns= dfm.columns.droplevel(1)
dfm.set_axis(['state_ansi','base','tavg_mean','tavg_std','temp_var_mean', 'temp_var_std','edd_mean','edd_std',
            'pcp_var_mean', 'pcp_var_std','pcp_mm_mean', 'pcp_mm_std','pcpdays_mean','pcpdays_std'], axis=1, inplace=True)

In [89]:
dfm

,state_ansi,base,tavg_mean,tavg_std,temp_var_mean,temp_var_std,edd_mean,edd_std,pcp_var_mean,pcp_var_std,pcp_mm_mean,pcp_mm_std,pcpdays_mean,pcpdays_std
0,01,0.0,23.248083,1.048496,4.337457,0.529232,4.954742,6.241053,8.790128,2.538451,675.777357,167.761223,5.903226,2.754358
1,01,1.0,23.804729,1.022763,4.237198,0.397912,9.865672,10.499922,9.206653,1.968270,677.448537,146.310761,6.280597,2.496479
2,01,2.0,23.552589,1.012194,3.994646,0.398881,2.880597,4.827945,8.887260,1.882628,736.090119,156.181126,6.504478,3.120218
3,01,3.0,23.707544,1.036497,4.338592,0.395342,3.119403,4.283803,9.717867,2.496457,777.499672,164.611784,6.713433,2.742704
4,04,0.0,22.804262,3.843348,5.042092,0.643167,31.131183,40.324091,2.442386,0.977076,162.941097,83.638385,0.225806,0.535970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,55,3.0,15.921609,1.271098,6.911132,0.635775,0.022222,0.147611,7.740859,1.560024,626.847500,127.615657,5.244444,2.536908
188,56,0.0,13.435515,2.163174,6.272045,0.749217,0.022440,0.241836,3.022736,0.893422,245.673436,73.621307,0.339411,0.646797
189,56,1.0,13.314336,2.327000,6.411802,0.501479,0.017391,0.131296,3.016861,1.021150,245.426087,84.853941,0.391304,0.733839
190,56,2.0,13.788186,1.967793,6.422604,0.730994,0.000000,0.000000,3.224451,0.830400,266.433478,63.078992,0.417391,0.675156


In [90]:
## Split dataframe: base versus period
dfb= dfm[dfm['base']==0.0]
dfm= dfm[dfm['base']>0.0]
dfb.rename(columns= {'tavg_mean': 'tavg_mean_base' ,'tavg_std': 'tavg_std_base',
                     'temp_var_mean':'temp_var_mean_base', 'temp_var_std': 'temp_var_std_base',
                     'edd_mean': 'edd_mean_base' ,'edd_std': 'edd_std_base',
                     'pcp_var_mean': 'pcp_var_mean_base', 'pcp_var_std': 'pcp_var_std_base',
                     'pcpdays_mean': 'pcpdays_mean_base', 'pcpdays_std': 'pcpdays_std_base',
                     'pcp_mm_mean': 'pcp_mm_mean_base', 'pcp_mm_std': 'pcp_mm_std_base'}, inplace=True)

dfb= dfb[['state_ansi','tavg_mean_base', 'tavg_std_base', 'temp_var_mean_base', 'temp_var_std_base',
          'edd_mean_base','edd_std_base', 'pcp_var_mean_base', 'pcp_var_std_base',
          'pcpdays_mean_base','pcpdays_std_base',
           'pcp_mm_mean_base', 'pcp_mm_std_base']]
dfm= pd.merge(dfm, dfb)
dfm['state_fips_code']= dfm['state_ansi']


In [91]:
dfm

,state_ansi,base,tavg_mean,tavg_std,temp_var_mean,temp_var_std,edd_mean,edd_std,pcp_var_mean,pcp_var_std,...,temp_var_std_base,edd_mean_base,edd_std_base,pcp_var_mean_base,pcp_var_std_base,pcpdays_mean_base,pcpdays_std_base,pcp_mm_mean_base,pcp_mm_std_base,state_fips_code
0,01,1.0,23.804729,1.022763,4.237198,0.397912,9.865672,10.499922,9.206653,1.968270,...,0.529232,4.954742,6.241053,8.790128,2.538451,5.903226,2.754358,675.777357,167.761223,01
1,01,2.0,23.552589,1.012194,3.994646,0.398881,2.880597,4.827945,8.887260,1.882628,...,0.529232,4.954742,6.241053,8.790128,2.538451,5.903226,2.754358,675.777357,167.761223,01
2,01,3.0,23.707544,1.036497,4.338592,0.395342,3.119403,4.283803,9.717867,2.496457,...,0.529232,4.954742,6.241053,8.790128,2.538451,5.903226,2.754358,675.777357,167.761223,01
3,04,1.0,23.239659,3.879533,5.182914,0.524754,34.586667,43.823076,2.214302,0.849031,...,0.643167,31.131183,40.324091,2.442386,0.977076,0.225806,0.535970,162.941097,83.638385,04
4,04,2.0,23.261857,3.890633,5.063232,0.446199,34.786667,43.953075,2.669576,1.010104,...,0.643167,31.131183,40.324091,2.442386,0.977076,0.225806,0.535970,162.941097,83.638385,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,55,2.0,15.777104,1.313094,6.272398,0.668791,0.041667,0.238233,7.636075,1.401781,...,0.587519,0.208333,0.765729,6.789385,1.403597,3.919803,2.201894,572.371165,113.375811,55
140,55,3.0,15.921609,1.271098,6.911132,0.635775,0.022222,0.147611,7.740859,1.560024,...,0.587519,0.208333,0.765729,6.789385,1.403597,3.919803,2.201894,572.371165,113.375811,55
141,56,1.0,13.314336,2.327000,6.411802,0.501479,0.017391,0.131296,3.016861,1.021150,...,0.749217,0.022440,0.241836,3.022736,0.893422,0.339411,0.646797,245.673436,73.621307,56
142,56,2.0,13.788186,1.967793,6.422604,0.730994,0.000000,0.000000,3.224451,0.830400,...,0.749217,0.022440,0.241836,3.022736,0.893422,0.339411,0.646797,245.673436,73.621307,56


In [92]:
## Compute standard deviations from the historical averages
for var in ['tavg', 'pcp_mm','temp_var', 'edd', 'pcp_var','pcpdays']:
    dfm[var+'_z']= (dfm[var+'_mean']- dfm[var+'_mean_base'])/dfm[var+'_std_base']

In [93]:
condlist= [(dfm['base']== 1.0),  
          (dfm['base']== 2.0), 
           (dfm['base']== 3.0)]
choicelist=[2012, 2017, 2022]
dfm['year'] = np.select(condlist, choicelist, np.nan)


In [94]:
dfm.columns

Index(['state_ansi', 'base', 'tavg_mean', 'tavg_std', 'temp_var_mean',
       'temp_var_std', 'edd_mean', 'edd_std', 'pcp_var_mean', 'pcp_var_std',
       'pcp_mm_mean', 'pcp_mm_std', 'pcpdays_mean', 'pcpdays_std',
       'tavg_mean_base', 'tavg_std_base', 'temp_var_mean_base',
       'temp_var_std_base', 'edd_mean_base', 'edd_std_base',
       'pcp_var_mean_base', 'pcp_var_std_base', 'pcpdays_mean_base',
       'pcpdays_std_base', 'pcp_mm_mean_base', 'pcp_mm_std_base',
       'state_fips_code', 'tavg_z', 'pcp_mm_z', 'temp_var_z', 'edd_z',
       'pcp_var_z', 'pcpdays_z', 'year'],
      dtype='object')

In [95]:
(dfm== np.inf).sum()

state_ansi            0
base                  0
tavg_mean             0
tavg_std              0
temp_var_mean         0
temp_var_std          0
edd_mean              0
edd_std               0
pcp_var_mean          0
pcp_var_std           0
pcp_mm_mean           0
pcp_mm_std            0
pcpdays_mean          0
pcpdays_std           0
tavg_mean_base        0
tavg_std_base         0
temp_var_mean_base    0
temp_var_std_base     0
edd_mean_base         0
edd_std_base          0
pcp_var_mean_base     0
pcp_var_std_base      0
pcpdays_mean_base     0
pcpdays_std_base      0
pcp_mm_mean_base      0
pcp_mm_std_base       0
state_fips_code       0
tavg_z                0
pcp_mm_z              0
temp_var_z            0
edd_z                 0
pcp_var_z             0
pcpdays_z             0
year                  0
dtype: int64

In [96]:
dfnoaa= dfm[['state_fips_code', 'year', 'tavg_mean', 'tavg_std', 'temp_var_mean',
       'edd_mean',  'pcp_var_mean', 'pcp_mm_mean', 'pcp_mm_std', 'pcpdays_mean',
       'tavg_z', 'pcp_mm_z', 'temp_var_z', 'edd_z', 'pcp_var_z', 'pcpdays_z']]

### Merge datasets

In [122]:
DF=pd.DataFrame()
DF= pd.merge(dfland1, dfprac, on=['year','state_fips_code','state_name','domaincat_desc'], how='left')

In [123]:
DF= pd.merge(DF, dfcrop, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') #insurance, rented land
DF= pd.merge(DF, dfirr, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') #irrigated cropland
DF= pd.merge(DF, dfy, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') # net income
DF= pd.merge(DF, dfg, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') # government programs
DF= pd.merge(DF, dfyf, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') # farm income
DF= pd.merge(DF, dfyf1, on=['year','state_fips_code','state_name','domaincat_desc'], how='left')# state programs
DF= pd.merge(DF, dfl, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') # labor expense
DF= pd.merge(DF, dfex, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') # expense total
DF= pd.merge(DF, dfmach, on=['year','state_fips_code','state_name','domaincat_desc'], how='left') #machinery assets
DF= pd.merge(DF, dfic, on=['year','state_fips_code','domaincat_desc'], how='left') # Main agro activities
DF= pd.merge(DF, dfnoaa, on=['year','state_fips_code'], how='left') #NOAA weather and climate


In [124]:
(DF.isna()).sum()

year                                          0
state_name                                    0
state_fips_code                               0
domaincat_desc                                0
ACRES OPERATED                                0
NUMBER OF OPERATIONS                          0
farmcat                                       0
lower_bound                                   0
upper_bound                                 144
(EXCL CRP) - ACRES                            4
DRAINED BY TILE - ACRES                       4
(EXCL CRP) - NUMBER OF OPERATIONS             4
DRAINED BY TILE - NUMBER OF OPERATIONS        4
CROP INSURANCE - ACRES                        6
CROP INSURANCE - NUMBER OF OPERATIONS         6
IRRIGATED - NUMBER OF OPERATIONS_x            9
RENTED FROM OTHERS_x                          5
RENTED FROM OTHERS_y                          5
OWNED_x                                       0
OWNED_y                                       0
IRRIGATED - ACRES                       

In [125]:
DF['farmcat']=DF['domaincat_desc'].str[16:-7]
DF.replace(np.nan, 0, inplace=True)
DF.replace('D', np.nan, inplace=True)


In [139]:
DF#.dropna(subset=['(EXCL CRP) - ACRES','DRAINED BY TILE - ACRES'], how='any')

,year,state_name,state_fips_code,domaincat_desc,ACRES OPERATED,NUMBER OF OPERATIONS,farmcat,lower_bound,upper_bound,(EXCL CRP) - ACRES,...,statepay_ope,labor_expense,labor_share,machinery_asset,grains_shareope,veggies_shareope,fruit_shareope,cattle_shareope,year_2017,year_2022
48,2012,ALABAMA,01,AREA OPERATED: (1.0 TO 9.9 ACRES),14379.0,2919,1-10,1,9.0,139.0,...,0.001028,0.077132,5.259637,36.973545,0.013703,0.079822,0.073998,0.220281,0,0
96,2012,ALABAMA,01,AREA OPERATED: (10.0 TO 49.9 ACRES),362197.0,13267,10-50,10,49.0,2651.0,...,0.003693,0.047568,3.586960,37.800758,0.024648,0.035803,0.036858,0.373785,0,0
432,2012,ALABAMA,01,AREA OPERATED: (50.0 TO 69.9 ACRES),246346.0,4235,50-70,50,69.0,1984.0,...,0.005195,0.040040,2.121030,34.715225,0.037308,0.016765,0.023849,0.452893,0,0
528,2012,ALABAMA,01,AREA OPERATED: (70.0 TO 99.9 ACRES),377435.0,4605,70-100,70,99.0,2921.0,...,0.007166,0.031832,1.831556,39.152023,0.029967,0.018458,0.022150,0.488599,0,0
144,2012,ALABAMA,01,AREA OPERATED: (100 TO 139 ACRES),500043.0,4342,100-140,100,139.0,3569.0,...,0.006909,0.036771,1.839224,30.717884,0.034086,0.019116,0.024413,0.463611,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,2022,WYOMING,56,AREA OPERATED: (220 TO 259 ACRES),55708.0,234,220-260,220,259.0,641.0,...,0.008547,0.076909,0.328539,7.886828,0.038462,0.000000,0.000000,0.363248,0,1
1583,2022,WYOMING,56,AREA OPERATED: (260 TO 499 ACRES),320538.0,887,260-500,260,499.0,2525.0,...,0.009019,0.105112,0.487556,8.916783,0.084555,0.001127,0.000000,0.394589,0,1
1679,2022,WYOMING,56,AREA OPERATED: (500 TO 999 ACRES),619090.0,882,500-1K,500,999.0,5269.0,...,0.001134,0.110806,1.464455,17.723672,0.071429,0.000000,0.001134,0.460317,0,1
1199,2022,WYOMING,56,"AREA OPERATED: (1,000 TO 1,999 ACRES)",1102112.0,798,1K-2K,1000,1999.0,6328.0,...,0.010025,0.084350,1.982757,22.068111,0.077694,0.000000,0.000000,0.527569,0,1


In [235]:
########## Region merge
df= pd.read_csv(cwd[:-4]+'/Data/data_farmclass_state.csv')
df['state_fips_code']= DF['state_fips_code'].astype(str).str.zfill(2)
list1= ['year', 'state_fips_code', 'domaincat_desc','state_abb','erosion_rate','State', 
        'Region', 'Region_fips','subregion','region_Midwest', 'region_Northeast', 'region_Pacific West',
       'region_Plains', 'region_Southeast', 'Class_Large', 'Class_Medium',
       'Class_Small', 'land_sqmi']
#DF.drop(['Unnamed: 0'], axis=1, inplace=True)
DF= pd.merge(DF, df[list1], on=['year','state_fips_code','domaincat_desc'])

In [ ]:
df= pd.read_csv(cwd[:-4]+'/Data/data_farmclass_state.csv')

### Land distribution

In [127]:
### Fill 'upper bound' with national mean values
# Replace missing values in Acres with the midpoint of the upper and lower bounds of the tabulated data of acres 
DF['upper_bound'].replace(0, 4000, inplace=True)
DF['ACRES OPERATED']= DF['ACRES OPERATED'].fillna((DF['upper_bound'])*(DF['NUMBER OF OPERATIONS']))


In [128]:
##### Share of cropland acres
DF['share_acres']= DF['ACRES OPERATED']/(DF.groupby(['year','state_fips_code'])['ACRES OPERATED'].transform('sum'))
# share of population (i.e %farms)
DF['share_farms']= DF['NUMBER OF OPERATIONS']/(DF.groupby(['year','state_fips_code'])['NUMBER OF OPERATIONS'].transform('sum'))

# Compute accumulative shares
DF= DF.sort_values(['year','state_fips_code','lower_bound'])
DF['acres_cum']= DF.groupby(['year','state_fips_code'])['share_acres'].cumsum()
DF['farms_cum']= DF.groupby(['year','state_fips_code'])['share_farms'].cumsum()

## Create a variable that measures the ratio: land share/operations (# farms) share
DF['land_concent']= DF['share_acres']/DF['share_farms']


In [138]:
DF.replace({'farmcat':{'1.0 TO 9.9': '1-10', '10.0 TO 49.9': '10-50', '50.0 TO 69.9': '50-70', '70.0 TO 99.9': '70-100', 
                     '100 TO 139': '100-140','140 TO 179': '140-180', '180 TO 219': '180-220', '220 TO 259': '220-260',
                     '260 TO 499': '260-500','500 TO 999':'500-1K', '1,000 TO 1,999': '1K-2K', '2,000 OR MORE': '>2K'}},inplace=True)
DF['class'] = DF['farmcat']
DF.replace({'class':{'1-10': 0, '10-50': 0, '50-70': 0, '70-100':0, 
                     '100-140': 1,'140-180': 1, '180-220': 1, '220-260': 1,
                     '260-500': 2, '500-1K': 2, '1K-2K': 2, '>2K': 2}},inplace=True)

DF['farm_class']= DF['class'].astype('category').map({0: 'Small', 1: 'Medium', 2: 'Large'})



### Adaptation practices

In [130]:
DF['cover_crops']= DF['(EXCL CRP) - ACRES']/DF['ACRES OPERATED']
DF['tile_ope']=DF['DRAINED BY TILE - NUMBER OF OPERATIONS']/DF['NUMBER OF OPERATIONS']
DF['irrigated_ope']= DF['IRRIGATED - NUMBER OF OPERATIONS_x']/DF['NUMBER OF OPERATIONS']

In [131]:
##### Proportion of rented land
DF['rented_land']= DF['RENTED FROM OTHERS_x']/(DF['OWNED_x']+DF['RENTED FROM OTHERS_x'])

#### Proportion of Insured land
DF['insured_farms']= DF['CROP INSURANCE - NUMBER OF OPERATIONS']/DF['NUMBER OF OPERATIONS']
DF['cropland_insured']= DF['CROP INSURANCE - ACRES']/DF['ACRES OPERATED']


### Farm Operations

In [132]:
## Create a variable 'new total operations' that takes the bigger number of operations between
# receiving income and federal transfers. 
# This variable will be used to determine the actual fed gov transf coverage with respect to farm income 
vals = []

for i, row in DF.iterrows():
    if row['FEDERAL - OPERATIONS WITH RECEIPTS'] > row['FARM-RELATED - OPERATIONS WITH RECEIPTS']:
        vals.append(row['FEDERAL - OPERATIONS WITH RECEIPTS'])
    else:
        vals.append(row['FARM-RELATED - OPERATIONS WITH RECEIPTS'])

DF['new_totoper'] = vals

In [133]:
### Income

DF.rename(columns= {'FARM-RELATED - RECEIPTS': 'tot_income'}, inplace=True)
DF.rename(columns= {'FARM-RELATED - OPERATIONS WITH RECEIPTS': 'income_ope'}, inplace=True)
### Fill 'upper bound' with national mean values
#Get mean income of large farms
DF['mean_farmincome']= DF['tot_income']/DF['income_ope']
# Get maximum average income in the farm class (national level)
DF['maxincome_category']= DF.groupby(['year','domaincat_desc'])['mean_farmincome'].transform('max')
# Fill upper bound
DF['mean_farmincome']= DF['mean_farmincome'].fillna(DF['maxincome_category'])
DF['tot_income']= DF['mean_farmincome']*DF['income_ope']

In [134]:

### Mean income
DF['netincome_perfarm']= DF['NET INCOME, MEASURED IN $ / OPERATION']

### Income from Government programs
DF['fed_income']= DF['FEDERAL - RECEIPTS']/DF['tot_income']

### State, local government receiving payments
DF['statepay_ope']= DF['STATE & LOCAL - OPERATIONS WITH RECEIPTS']/DF['NUMBER OF OPERATIONS']
       
### Hired Labor Expense 
DF['labor_expense']= (DF['HIRED - EXPENSE']+DF['CONTRACT - EXPENSE'])/DF['EXPENSE, MEASURED IN $']

DF['labor_share']= (DF['HIRED - EXPENSE']+DF['CONTRACT - EXPENSE'])/DF['tot_income']

### Technology in Assets
DF['machinery_asset']= (DF['MACHINERY TOTALS_x'])/DF['tot_income']


In [135]:
### Agricultural practices NAICS

for x in ['grains', 'veggies', 'fruit', 'cattle']:
    DF[x+'_shareope']= DF[x]/DF['NUMBER OF OPERATIONS']

In [62]:
##############
# Define farm-region categories
DF['farm_region']= DF[['farm class','Region']].apply('-'.join, axis=1)
DF['farm_region'].replace(to_replace=['Small-Pacific West','Small-Plains','Small-Midwest','Small-Southeast','Small-Northeast',
                                     'Medium-Pacific West','Medium-Plains','Medium-Midwest','Medium-Southeast','Medium-Northeast',
                                     'Large-Pacific West','Large-Plains','Large-Midwest','Large-Southeast','Large-Northeast'],
                         value=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14], inplace=True)

DF['region']= DF['Region']
DF= pd.get_dummies(DF, columns=['region'])

DF['Class']= DF['farm_class']
DF= pd.get_dummies(DF, columns=['Class'])
DF['state_code']= DF['state_fips_code']
DF= pd.get_dummies(DF, columns=['state_code'])

# Impute maximum values on outliers
DF.loc[DF.insured_farms > 1, 'insured_farms'] = 1
DF.loc[DF['cover crops']> 0.4, 'cover crops']= 0.4

## Agricultural harvested cropland as % total state land
DF['cropland_share']= DF['acres_tot']/(DF['land_sqmi']*640)

In [136]:
# Year 2017 dummy
DF['year_2017'] = np.where(DF['year']==2017, 1, 0)
DF['year_2022'] = np.where(DF['year']==2022, 1, 0)

In [119]:
DF['tile_acres']=DF['DRAINED BY TILE - ACRES']/DF['ACRES OPERATED']
DF['irrigated_acres']= DF['IRRIGATED - ACRES']/DF['ACRES OPERATED']

In [120]:
DF.to_csv(cwd[:-4]+'/Data/data_farmclass_state.csv', index=False)